<a href="https://colab.research.google.com/github/yukuproj/made2/blob/main/hw2_kuznetsov_ya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Что? Где? Когда? Рейтинги homework 2




In [1]:
import numpy as np
import pandas as pd
import pickle
import re
import scipy.stats as stats


from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from scipy import sparse as sp
from scipy.special import expit


In [2]:
ls

drive/  sample_data/


In [47]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Read

In [3]:
players_file = "/content/drive/MyDrive/made2_ml_hw2/players.pkl"
results_file = "/content/drive/MyDrive/made2_ml_hw2/results.pkl"
tournaments_file = ("/content/drive/MyDrive/made2_ml_hw2/tournaments.pkl")

with open(players_file, 'rb') as fin:
     players_data = pickle.load(fin)

with open(tournaments_file, 'rb') as fin:
     tournaments_data = pickle.load(fin)

with open(results_file, 'rb') as fin:
     results_data = pickle.load(fin)     

Filtering data

In [6]:
def isOk(tr):
    rez1 = tr.get('mask') and tr.get('teamMembers')    
    rez2 = len(tr.get('teamMembers')) > 0    
    rez3 = all(players_data.get(tm['player']['id']) for tm in tr.get('teamMembers'))
    if not(rez1 and rez2 and rez3):  ##  нет состава команд или детализации по вопросам
      return False
    rez4 = not re.findall('[^01]', tr.get('mask'))   ## нет четких ответов
    return rez4

In [7]:
results_filtered = {}
tournaments_filtered = {}

for t_id, team_results in results_data.items():
    team_results_filtered = [tr for tr in team_results if isOk(tr)]
    if (len(team_results_filtered) > 0):
        results_filtered[t_id] = team_results_filtered
        tournaments_filtered[t_id] = tournaments_data[t_id]

In [8]:
tournaments_train = {k: v for k, v in tournaments_filtered.items() if v['dateStart'][:4] == '2019'}  ## тестовая выборка -турниры за 2020 год
tournaments_test  = {k: v for k, v in tournaments_filtered.items() if v['dateStart'][:4] == '2020'}  ## обучающая выборка - турниры за 2019 год

In [9]:
print(f'Размер обучающей выборки: {len(tournaments_train)}')
print(f'Размер тестовой выборки:  {len(tournaments_test)}')

Размер обучающей выборки: 616
Размер тестовой выборки:  160


## Baseline-модель <a name = "baseline"/>

Будем предполагать что игрок отвечает точно также как и его команда. Т.е. если команда на вопрос ответила, то это значит, что и каждый игрок внутри команды тоже способен был самостоятельно ответить на вопрос. И обратно, более естественное требование, если команда не ответила, то не смог бы ответить и каждый игрок. 

Будем считать что модель устроена как логистическая регрессия, в которой в качестве аргументов являются метки игрока и метка вопроса. Неизвестными коэффициентами в этой регрессии будут являться "уровень" игрока и "сложность вопроса". При этом обучаться будем на 0 или 1 в зависимости от того, ответил ли игрок на вопрос или нет.

Для определения переменных модели закодируем игрока и вопрос в с помощью OneHotEncoding представления. Входной вектор для обучения тогда - это вектор из нулей и единиц, где единицы соответствуют конкретному игроку и конкретному вопросу на который он отвечает, а ответ - 0 или 1 в зависимости от того, ответил ли игрок на вопрос. 




In [ ]:

max_question_id = 0

train_table = []

for tournament_id, tournament in tqdm(tournaments_train.items()):
    for team_result in results_filtered[tournament_id]:
        team_id = team_result['team']['id']
        mask = np.array([np.int32(answer) for answer in team_result['mask']])
        players = team_result['teamMembers']
        questions = np.arange(max_question_id, max_question_id + len(mask))
        
        for player in players:
            player_id = player['player']['id']
            for i in range(len(mask)):
                train_table.append([tournament_id, team_id, player_id, questions[i], mask[i]])
    max_question_id += len(mask)    
        
train_df = pd.DataFrame(train_table, 
                        columns = ['tournament_id', 'team_id', 'player_id', 'question_id', 'answer'])      

100%|██████████| 616/616 [00:38<00:00, 16.06it/s]


In [ ]:
ohe = OneHotEncoder(handle_unknown='ignore')
X_train = ohe.fit_transform(train_df[['player_id', 'question_id']])
y_train = train_df['answer']

In [ ]:
lr = LogisticRegression(random_state=random_state, n_jobs=-1)
lr.fit(X_train, y_train)

LogisticRegression(n_jobs=-1, random_state=42)

In [ ]:
active_players = np.unique(train_df['player_id'])
rating = pd.DataFrame({'id': active_players,
                       'coef': lr.coef_[0][:len(active_players)],
                       'name': [(players_data[i]['name'] + ' ' + players_data[i]['surname']) for i in active_players]
                      })

In [ ]:
rating.sort_values(by='coef', ascending=False).head(200)

,id,coef,name
3803,27403,3.589675,Максим Руссо
593,4270,3.537738,Александра Брутер
5106,37047,3.384659,Мария Юнгер
3991,28751,3.311466,Иван Семушин
5289,38196,3.247600,Артём Митрофанов
...,...,...,...
7453,66764,2.422163,Эльмира Гулуева
2719,19468,2.420859,Виктория Маландина
1287,9188,2.417401,Ольга Деркач
650,4730,2.417401,Владислав Быков


## Оценка качества <a name = "quality"/>

Отранжировать команды при оценке качества на тестовой выборке можно, исходя из оценки вероятности того, что хотя бы один участник в команде отвечает на вопрос.

In [ ]:
test_table = []

for tournament_id, tournament in tqdm(tournaments_test.items()):
    for team_result in results_filtered[tournament_id]:
        team_id = team_result['team']['id']
        mask = np.array([np.int32(answer) for answer in team_result['mask']])
        players = team_result['teamMembers']
        
        for player in players:
            player_id = player['player']['id']
            if player_id not in active_players:
                continue
            test_table.append([tournament_id, team_id, player_id, -1, sum(mask), len(mask)])  
        
test_df = pd.DataFrame(test_table, 
                        columns = ['tournament_id', 'team_id', 'player_id', 'question_id', 'correct_answers', 'total_answers'])      

100%|██████████| 160/160 [00:04<00:00, 33.59it/s]


In [ ]:
X_test = test_df[['player_id', 'question_id']]
X_test = ohe.transform(X_test)
predictions = lr.predict_proba(X_test)[:, 1]

In [ ]:
def compute_scores(data, preds):
    data['pred'] = preds
    data['score'] = data.groupby(['tournament_id', 'team_id'])['pred'].transform(lambda x: 1 - np.prod(1 - x))
    rating = data[['tournament_id', 'team_id', 'correct_answers', 'score']].drop_duplicates().reset_index(drop=True)
    
    # Считаем реальный рейтинг команд
    rating = rating.sort_values(by=['tournament_id', 'correct_answers'], ascending=False)
    rating['real_rank'] = rating.groupby('tournament_id')['correct_answers'].transform(lambda x: np.arange(1, len(x) + 1))
    
    # Считаем предсказанный рейтинг
    rating = rating.sort_values(by=['tournament_id', 'score'], ascending=False)
    rating['pred_rank'] = rating.groupby('tournament_id')['score'].transform(lambda x: np.arange(1, len(x) + 1))

    rating = rating.astype(np.int32)
    
    print(f"Корреляция Спирмана: {rating.groupby('tournament_id').apply(lambda x: stats.spearmanr(x['real_rank'], x['pred_rank']).correlation).mean()}")
    print(f"Корреляция Кендалла: {rating.groupby('tournament_id').apply(lambda x: stats.kendalltau(x['real_rank'], x['pred_rank']).correlation).mean()}")

In [ ]:
compute_scores(test_df, predictions)

Корреляция Спирмана: 0.7672515187411473
Корреляция Кендалла: 0.5987298826719607


## EM алгоритм <a name = "EM"/>

Усовершенствуем модель. 

Прежде всего необходимо учитывать то, что на вопрос отвечают сразу несколько игроков. В baseline мы считали, что если команда ответила на вопрос, то и каждый игрок в команде ответил на вопрос. Однако в реальности это только дает нам право говорить, что если команда не ответила на вопрос, то ни один игрок в команде на него ответил.
А так же, если хотя бы один игрок ответил на вопрос, то и команда ответила


Введем новые обозначения

Пусть 
$A$ - событие, игрок ответил на вопрос
$B$ - команда ответила на вопрос

Тогда можем вывести следующие соотношения:

$P(B|A) = 1
\\
P(A|\overline{B}) = 0
\\
P(A|B) = \frac{P(B|A)P(A)}{P(B)} = \frac{P(A)}{P(B)}
\\
P(B) = 1 - \prod(1-P(A))
$

Таким образом реализация EM-алгоритма будет выглядеть следующим образом:

E-step - оценка $P(A|B)$

M-step - обучение логистической регрессии на таргете с E-step

В итоге получаем $P(A)$

In [ ]:
def log_loss(y_true, y_pred):
    return - np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

class EMClassifier:
    
    def __init__(self, w=None, lr=15, n_iter=10, batch_size=5000, verbose=1):
        self.w = w
        self.lr = lr
        self.n_iter = n_iter
        self.batch_size = batch_size
        self.verbose = 1
        
    def _add_intercept(self, X):
        return sp.hstack((np.ones((X.shape[0], 1)), X), format='csr')
    
        
    def _E_step(self, data, preds):
        team_prob = pd.DataFrame({'team_id': data['team_id'],
                                      'question_id': data['question_id'], 
                                      'team_prob': 1 - preds})\
                        .groupby(['team_id', 'question_id']).agg({'team_prob': 'prod'}).reset_index()

        team_prob['team_prob'] = 1 - team_prob['team_prob']
        team_prob = data[['team_id', 'question_id']].merge(team_prob)
        y = np.clip(preds / team_prob['team_prob'], 0, 1).values
        y[data['answer'] == 0] = 0
        return y
        
    def _M_step(self, X, y):
        min_loss = np.inf
        indices = np.arange(X.shape[0])
        for _ in range(100):
            indices = np.random.permutation(indices)
            for batch_idx in np.array_split(indices, len(indices) // self.batch_size):
                x_batch, y_batch = X[batch_idx], y[batch_idx]
                grad = x_batch.T.dot(self.predict(x_batch) - y_batch) / len(y_batch)
                self.w -= self.lr * grad
                
            cur_loss = log_loss(y, self.predict(X))
            if min_loss - cur_loss < 1e-6:
                break
                
            min_loss = cur_loss
                
    def fit(self, X_tr, train_data, X_te=None, test_data=None):
        X_tr = self._add_intercept(X_tr)
        for iter_ in tqdm(range(self.n_iter)): 
            preds = self.predict(X_tr)
            y = self._E_step(train_data, preds)
            self._M_step(X_tr, y)
            if self.verbose is not None and X_te is not None and test_data is not None and iter_ % self.verbose == 0:
                compute_scores(test_data, self.predict(X_te))
                         
    def predict(self, X):
        if len(self.w) != X.shape[1]:
            X = self._add_intercept(X)
        return expit(X.dot(self.w))

In [ ]:
# Веса инициализирются предобученной baseline моделью
w_init = np.hstack([lr.intercept_, lr.coef_[0]])
em_classifier = EMClassifier(w_init)

em_classifier.fit(X_train, train_df, X_test, test_df)

 10%|█         | 1/10 [01:53<17:05, 113.95s/it]

Корреляция Спирмана: 0.776488395036184
Корреляция Кендалла: 0.6090761165465001


 20%|██        | 2/10 [07:22<23:45, 178.20s/it]

Корреляция Спирмана: 0.7756330734582625
Корреляция Кендалла: 0.608241723690095


 30%|███       | 3/10 [11:15<22:42, 194.63s/it]

...

Корреляция Спирмана: 0.7773127277069969
Корреляция Кендалла: 0.6092628292823374


 90%|█████████ | 9/10 [42:56<03:57, 237.13s/it]

Корреляция Спирмана: 0.7774699920471461
Корреляция Кендалла: 0.6095793760849213


100%|██████████| 10/10 [46:35<00:00, 279.50s/it]

Корреляция Спирмана: 0.7780886247425316
Корреляция Кендалла: 0.610170353216916


По результатам обучения наблюдается прирост качества, пусть и небольшой.

## Рейтинг турниров <a name = "rating_tournaments"/>

А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов. Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть сложные вопросы, а на турнирах для школьников — простые)? Если будет интересно: постройте топ сложных и простых вопросов со ссылками на конкретные записи в базе вопросов ЧГК (это чисто техническое дело, тут никакого ML нету).

Сложность турнира можно посчитать как среднюю сложность вопросов в турнире - возьмем средние коэффициенты обученной модели.

In [ ]:
active_questions = np.unique(train_df['question_id'])
q_rating = dict(zip(active_questions, em_classifier.w[-len(active_questions):]))

train_df['difficulty'] = train_df['question_id'].map(q_rating)
train_df['tournament_name'] = train_df['tournament_id'].map({v['id']: v['name']for k, v in tournaments_data.items()})

tournaments_rating = train_df[['tournament_name', 'question_id', 'difficulty']].drop_duplicates()
tournaments_rating = tournaments_rating.groupby('tournament_name')['difficulty'].mean().sort_values().reset_index()

Ниже расположены рейтинг самых сложных турниров

In [ ]:
tournaments_rating.head(20)

,tournament_name,difficulty
0,Чемпионат Санкт-Петербурга. Первая лига,-3.593347
1,Угрюмый Ёрш,-2.033586
2,Первенство правого полушария,-1.848786
3,Кубок городов,-1.621134
4,Воображаемый музей,-1.554841
5,Записки охотника,-1.459090
6,Чемпионат России,-1.458811
7,Ускользающая сова,-1.411516
8,All Cats Are Beautiful,-1.409829
9,VERSUS: Коробейников vs. Матвеев,-1.393327


## Рейтинг игроков <a name = "rating_players"/>

Посмотрим еще раз рейтинг игроков

In [ ]:
rating = pd.DataFrame({'id': active_players,
                       'coef': em_classifier.w[1:1 + len(active_players)],
                       'name': [(players_data[i]['name'] + ' ' + players_data[i]['surname']) for i in active_players]
                      })
rating['questions_count'] = rating['id'].map(train_df.groupby('player_id')['question_id'].count())

In [ ]:
rating.sort_values(by='coef', ascending=False).head(30)

,id,coef,name,questions_count
56810,222188,3.924734,Арина Гринко,216
3803,27403,3.763272,Максим Руссо,2075
9199,87637,3.574147,Антон Саксонов,1179
54137,216863,3.550071,Глеб Гаврилов,252
8063,74001,3.521446,Игорь Мокин,1071
593,4270,3.410755,Александра Брутер,2555
3844,27622,3.391753,Николай Рябых,321
4176,30152,3.296343,Артём Сорожкин,4375
3396,24384,3.271007,Евгений Пашковский,1629
3991,28751,3.270155,Иван Семушин,3386


Среди игроков в топе появилось много игроков, у которых довольно мало сыгранных турниров. Попробуем брать в рейтинг только тех игроков у которых сыграно больше 1000 игр.


In [ ]:
rating[rating['questions_count'] > 1000].sort_values(by='coef', ascending=False).head(30)

,id,coef,name,questions_count
3803,27403,3.763272,Максим Руссо,2075
9199,87637,3.574147,Антон Саксонов,1179
8063,74001,3.521446,Игорь Мокин,1071
593,4270,3.410755,Александра Брутер,2555
4176,30152,3.296343,Артём Сорожкин,4375
3396,24384,3.271007,Евгений Пашковский,1629
3991,28751,3.270155,Иван Семушин,3386
4800,34846,3.239409,Антон Чернин,1751
4196,30270,3.203064,Сергей Спешков,3440
3874,27822,3.191496,Михаил Савченков,3107


Теперь в топ попали действительно хорошие игроки, находящиеся в топе на сайте рейтинга.